In [65]:
from pickle import TRUE
import dataSplit
import os, shutil
import redundancy
import Stockholm_Fasta
import fileNumber
import PID_save as pid
import redundancy
import numpy as np
import pandas as pd
import mainBlosum
import mainBlosumBrier
import lowerToUpper
import dataCountDescription
import mainNeighbor
import ContextuelBrier

# Initial configuration

Manual initialisation of a folder containing Pfam_dev (sample of the downloaded version from Pfam)

In [66]:
path_main_folder =  "/Users/pauline/Desktop/test_dev" 
name_file_from_Pfam = "Pfam_dev"
path_file_from_Pfam = f"{path_main_folder}/{name_file_from_Pfam}" 


#  User choices

In [67]:
name_folder_stockholm = "Pfam_Stockholm"
name_folder_fasta = "Pfam_fasta"
name_folder_upper = "Pfam_upper"
name_folder_pid = "PID_couple"
name_fasta_non_redondant = "Pfam_fasta_99"

# à enlever script list_AA <-- list_residu
list_residu = ["A", "R", "N", "D", "C", "Q", "E", "G", "H", "I", "L", "K", "M", "F", "P", "S", "T", "W", "Y", "V"]
pid_sup = 99

In [68]:
data_treatment = True
descriptionPfam = False

## Data treatment
(do it once !)

In [72]:
path_folder_stockholm = f"{path_main_folder}/{name_folder_stockholm}"  
path_folder_fasta = f"{path_main_folder}/{name_folder_fasta}"
path_folder_upper = f"{path_main_folder}/{name_folder_upper}"
path_folder_pId = f"{path_main_folder}/{name_folder_pid}"
path_folder_fasta_non_redondant =  f"{path_main_folder}/{name_fasta_non_redondant}" 

if data_treatment == True:
    # separation of the Stockholm file into Stockholm files
    Stockholm_Fasta.separationStockholm(path_file_from_Pfam, path_folder_stockholm) 
    #fileNumber.countFile(path_folder_stockholm)  

    # conversion from stockholm into fasta files
    Stockholm_Fasta.multiStockholmToFasta(path_folder_fasta, path_folder_stockholm)  
    #dataCountDescription.dataCountDescription(path_folder_fasta)

    # conversion of all the residus lower case into upper case
    lowerToUpper.multiLowerToUpper(path_folder_fasta, path_folder_upper)
    #dataCountDescription.dataCountDescription(path_folder_upper)

    # pid couple calculation
    pid.savePId(path_folder_upper, path_folder_pId)   

    # redundancy issue  
    redundancy.savePIdNonRedondant(path_folder_upper, path_folder_fasta_non_redondant, pid_sup, list_residu) 
    #dataCountDescription.dataCountDescription(path_folder_fasta_non_redondant)


# description Pfam after data treatment
if descriptionPfam == True:
    dataCountDescription.dataCountDescription(path_folder_fasta_non_redondant)




--- Separation of the multiStockholm file into monoStockholm files: 0.01249 s
--- Conversion of Stockholm files into Fasta files: 0.05478 s
--- Correction upper files: 0.01139 s
--- Compute and save the pId files: 2.36516 s
--- Compute and save non-redundant files: 1.14333 s
